In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import sys
import os
from zipfile import ZipFile
import csv
import random
from math import isnan 

In [2]:
scyjava.config.add_option('-Xmx30g')
#ij = imagej.init('/home/saka/fiji-linux64/Fiji.app', mode = 'interactive')
ij = imagej.init('/home/saka/sw/local/fiji/2023', mode='interactive')
ij.ui().showUI()
ij.getVersion()

'2.14.0/1.54f'

Unexpected exception formatting exception. Falling back to standard exception
['61.roi', '207.roi', '249.roi', '113.roi', '394.roi', '321.roi', '107.roi', '28.roi', '375.roi', '384.roi', '251.roi', '391.roi', '11.roi', '222.roi', '67.roi', '4.roi', '278.roi', '37.roi', '99.roi', '358.roi', '339.roi', '163.roi', '280.roi', '382.roi', '304.roi', '45.roi', '397.roi', '68.roi', '256.roi', '309.roi', '48.roi', '296.roi', '149.roi', '70.roi', '98.roi', '35.roi', '432.roi', '140.roi', '139.roi', '347.roi', '243.roi', '405.roi', '191.roi', '281.roi', '223.roi', '105.roi', '25.roi', '87.roi', '180.roi', '303.roi', '10.roi', '377.roi', '55.roi', '71.roi', '153.roi', '380.roi', '271.roi', '59.roi', '86.roi', '214.roi', '129.roi', '241.roi', '170.roi', '379.roi', '374.roi', '209.roi', '299.roi', '1.roi', '172.roi', '134.roi', '333.roi', '65.roi', '291.roi', '290.roi', '27.roi', '51.roi', '126.roi', '265.roi', '253.roi', '312.roi', '174.roi', '418.roi', '371.roi', '368.roi', '176.roi', '361.roi', '

In [3]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
Duplicator = scyjava.jimport('ij.plugin.Duplicator')()
ov = Overlay()
Model =  scyjava.jimport('fiji.plugin.trackmate.Model')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')

rm = ij.RoiManager.getRoiManager()

In [4]:
def track_bacteria(bact, directory_path, dsettings, quality_filter, tsettings, display=False):
    '''
    This function leverages ImageJ's Trackmate plugin to track the bacteria within each single-cell video provided in the input folder bact. 
    It uses the Tresholding Detector for segmentation (with detection parameters specified in dsettings), and the LAP Tracker algorithm for tracking (with tracking parameters specified in tsettings). The function also takes as input the desired quality filter on the detected spots.
    The function saves the Trackmate XML model as well as a CSV file of the tracks for each cell in bact/Output/.
    '''
    out = bact_path+"Output/"
    if not os.path.exists(out):
        os.makedirs(out)
    for image in os.listdir(bact):
        if (image[len(image)-4:] == ".tif"):
            # Open Image
            imp = ij.IJ.openImage(bact + image)
            imp.show()
            ij.IJ.run("Smooth", "stack")
            ij.IJ.run("Smooth", "stack")
            # Create Model
            model = Model()
            settings = Settings(imp)
            # Detector
            settings.detectorFactory = ThresholdDetectorFactory()
            for parameter, value in dsettings.items():
                settings.detectorSettings[parameter] = value
            filter1 = FeatureFilter('QUALITY', quality_filter, True)
            settings.addSpotFilter(filter1)
            print(settings.detectorSettings)
            # Tracker
            settings.trackerFactory = SparseLAPTrackerFactory()
            settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
            for parameter, value in tsettings.items():
                settings.trackerSettings[parameter] = value
            # Execute Tracking
            trackmate = TrackMate(model, settings)
            ok = trackmate.checkInput()
            if not ok:
                sys.exit(str(trackmate.getErrorMessage()))
            ok = trackmate.process()
            if not ok:
                sys.exit(str(trackmate.getErrorMessage()))
            selectionModel = SelectionModel(model)
            # Display
            ds = DisplaySettingsIO.readUserDefault()
            if display:
                displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
                displayer.render()
                displayer.refresh()
                trackscheme = TrackScheme(model, selectionModel, ds)
                trackscheme.render()
            # Save Data
            basename_image = os.path.basename(image)
            corename = os.path.splitext(basename_image)[0]
            outFile = Path(out+corename+"_exportModel.xml")
            writer = TmXmlWriter(outFile)
            writer.appendModel(model)
            writer.appendSettings(settings)
            writer.writeToFile()
            csvFileSpots = out+corename+"_exportTracks.csv"
            spotsTableView = AllSpotsTableView(model, selectionModel, ds)
            spotsTableView.exportToCsv(csvFileSpots)
    ij.py.run_macro('close("*")')
    
def merge_trackmate_rois(bact_path, directory_path):
    '''
    This function seeks to assign bacterial ROIs to tracks by merging the results of tracking (via Trackmate) and ROI detection (via Thresholding in ImageJ). It matches ROIs to spots detected in Trackmate based on the nearest centroid.
    For each cell, the function saves a CSV file of tracks and their corresponding ROIs in a folder called grouped.
    '''
    file_pattern = os.path.join(bact_path, "*.tif")
    file_list = glob.glob(file_pattern)
    out = directory_path+"grouped/"
    if not os.path.exists(out):
        os.makedirs(out)
    for file_path in file_list:
        filename = os.path.basename(file_path)
        filename2 = os.path.splitext(filename)[0]
        first = Path(f"{bact_path}/Output/{filename2}_exportTracks.csv").as_posix()
        second = Path(f"{bact_path}/{filename2}_bactROI.csv").as_posix()
        pairwise = xref_locations(first, second, 
                              first_x='POSITION_X', 
                              first_y='POSITION_Y', 
                              first_z='POSITION_Z', 
                              second_x='X', 
                              second_y='Y', 
                              second_z='Slice', 
                              verbose=False)
        grouped = pairwise.groupby('ID', group_keys=False)[pairwise.columns[12]].apply(list).reset_index()
        grouped.rename(columns={'index_right': 'object_ID_list'}, inplace=True)
        final_csv = Path(f"{directory_path}/grouped/{filename2}_grouped.csv")
        grouped.to_csv(final_csv)
        final_csv2 = Path(f"{directory_path}/grouped/{filename2}_pairwise.csv")
        pairwise.to_csv(final_csv2)
    return out

def apply_bact_overlays(bact_path, grouped):
    '''
    This function allows for visual quality-control of the bacterial-tracking step by adding colored overlays to the bacterial tracks in each single-cell video. Bacterial clusters of the same track remain the same color from frame to frame.
    All of the overlayed videos are saved in the folder bact with the title [cell_number]_overlay.tif.
    '''
    file_pattern = os.path.join(bact_path, "*.tif")
    file_list = glob.glob(file_pattern)
    colors = ["blue", "cyan", "green", "magenta", "orange", "red", "yellow"]
    for file_path in file_list: # cycle through images, open them and open the matching "grouped' csv file
        imp = ij.IJ.openImage(file_path)
        ij.ui().show(imp)
        f_name = os.path.basename(file_path)
        f_name = os.path.splitext(f_name)[0]
        input_csv = Path(f"{grouped}/{f_name}_grouped.csv")
        df = pandas.read_csv(input_csv)
        pouet = df['object_ID_list']
        input_ROI = Path(f"{bact_path}/{f_name}_bactROI.zip")
        rm.open(f"{input_ROI}")
        for i in range(len(pouet)):
            single_row = df.iloc[i]
            random_color = random.choice(colors)
            row_cellids = single_row.object_ID_list
            row_cleaned = row_cellids.strip('[').strip(']')
            row_array = row_cleaned.split(', ')
            for cell in row_array:
                cell_index = int(cell)
                roi = rm.select(cell_index)
                overlay_command = f"Overlay.addSelection('{random_color}',2);"
                ij.py.run_macro(overlay_command)
        ij.py.run_macro("setMinAndMax(0, 100);")
        ij.py.run_macro("run('Flatten', 'stack');")
        method = 'max all'
        z_projector_result = ZProjector.run(imp, method)
        z_collapsed_image = ij.py.from_java(z_projector_result)
        z_collapsed_dataset = ij.py.to_dataset(z_projector_result)
        result_path = os.path.splitext(file_path)[0] + "_overlay.tif"
        ij.io().save(z_collapsed_dataset, result_path)
        ij.py.run_macro("close();")
        ij.py.run_macro("roiManager('Select All');")
        rm.runCommand("Delete")

def xref_locations(first, second, first_x='POSITION_X', first_y='POSITION_Y', first_z='POSITION_Z',
                   second_x='X', second_y='Y', second_z='Slice',
                   max_dist=20, verbose=False):
    pairwise_elements = pandas.DataFrame()
    first_measurements = pandas.read_csv(first)
    first_measurements = first_measurements.drop([0,1,2])
    second_measurements = pandas.read_csv(second)
    first_gdf = geopandas.GeoDataFrame(
        first_measurements,
        geometry=geopandas.points_from_xy(first_measurements[first_x],
                                          first_measurements[first_y],
                                          first_measurements[first_z]))
    second_gdf = geopandas.GeoDataFrame(
        second_measurements,
        geometry=geopandas.points_from_xy(second_measurements[second_x],
                                          second_measurements[second_y],
                                          second_measurements[second_z]))
    ti_rows = first_gdf.shape[0]
    tj_rows = second_gdf.shape[0]
    for ti_row in range(0, ti_rows):
        if verbose:
            print(f"On row: {ti_row}")
        ti_element = first_gdf.iloc[[ti_row, ]]
        
        titj = geopandas.sjoin_nearest(ti_element, second_gdf,
                                       distance_col="pairwise_dist",
                                       max_distance=max_dist)
        chosen_closest_dist = titj.pairwise_dist.min()
        if (isnan(chosen_closest_dist)):
            print(f"This element has no neighbor within {max_dist}.")
        else:
            chosen_closest_cell = titj.pairwise_dist == chosen_closest_dist
            chosen_closest_row = titj[chosen_closest_cell]
            pairwise_tmp = pandas.concat([pairwise_elements, chosen_closest_row])
            pairwise_elements = pairwise_tmp
    return pairwise_elements

In [6]:
#define your parent directory
directory_path = '/home/saka/Documents/Lab_stuff/confocal/20230427 thp1lc3 gal3 alix/gal3/'

#Creation of different directory for outputs
bact_path = directory_path + "bact/"
if not os.path.exists(bact_path):
    os.makedirs(bact_path)
measurement_path = directory_path + "measurement/"
if not os.path.exists(measurement_path):
    os.makedirs(measurement_path)
channels_path = directory_path + "channels/"
if not os.path.exists(channels_path):
    os.makedirs(channels_path)
voronoi_path = directory_path + "voronoi/"
if not os.path.exists(voronoi_path):
    os.makedirs(voronoi_path)

In [ ]:
format = f'Tiff'
file_pattern = os.path.join(directory_path, "*.czi")
file_list = glob.glob(file_pattern)
for file_path in file_list:
    raw_image = image = ij.io().open(file_path)
    basename = os.path.basename(file_path)
    corename = os.path.splitext(basename)[0]
    LC3_channel = 1
    channel = raw_image[:, :, LC3_channel, :]
    lc3 = ij.py.to_imageplus(channel)
    lc3.setDimensions(1, 11, 1)
    #ij.ui().show(lc3)
    result_path = f"{channels_path}/{corename}_LC3_channel.tif"
    ij.IJ.saveAs(lc3, "Tiff", ij.py.to_java(result_path))
    
    gal3_channel = 3
    channel = raw_image[:, :, gal3_channel, :]
    gal3 = ij.py.to_imageplus(channel)
    gal3.setDimensions(1, 11, 1)
    result_path = f"{channels_path}/{corename}_gal3_channel.tif"
    ij.IJ.saveAs(gal3, "Tiff", ij.py.to_java(result_path))
    
    bact_channel = 2
    channel = raw_image[:, :, bact_channel, :]
    bact = ij.py.to_imageplus(channel)
    bact.setDimensions(1, 11, 1)
    result_path = f"{bact_path}/{corename}_bact_channel.tif"
    ij.IJ.saveAs(bact, "Tiff", ij.py.to_java(result_path))

    method = 'max'
    nuclei_channel =0
    blue_channel = raw_image[:, :, nuclei_channel,:]
    sigma = 1.5  # Adjust the value of sigma as needed
    smoothed_image = ij.op().run("smooth", blue_channel, sigma)
    imp = ij.py.to_imageplus(smoothed_image) # convert the image into the imageplus type object needed for z-projection
    # Z-projection
    z_projector_result = ZProjector.run(imp, method)
    z_collapsed_image = ij.py.from_java(z_projector_result)
    z_collapsed_dataset = ij.py.to_dataset(z_collapsed_image)  
    # Save as a TIFF imagerm.open(f"{input_roi}")
    result_path = f"{voronoi_path}/{corename}_max_nuclei_channel.tif"
    ij.io().save(z_collapsed_dataset, result_path)
    print(f"Saving image {result_path}.")

In [ ]:
file_pattern = os.path.join(voronoi_path, "*nuclei_channel.tif")
file_list = glob.glob(file_pattern)
model = models.CellposeModel(gpu=True, model_type='THP1_nuclei')
for file_path in file_list:
    imgs = io.imread(file_path)
    channels = [[0,0]]
    masks, flows, styles = model.eval(imgs, diameter=None, channels=channels)
    io.save_to_png(imgs, masks, flows, file_path)

In [ ]:
for file_path in file_list:
    image_red = ij.io().open(file_path)
    imp = ij.py.to_imageplus(image_red)
    f_name = os.path.basename(file_path)
    f_name = os.path.splitext(f_name)[0]
    input_txt = Path(f"{voronoi_path}/{f_name}_cp_outlines.txt")
    txt_fh = open(input_txt, 'r')
    for line in txt_fh:
        xy = line.rstrip().split(",")
        xy_coords = [int(element) for element in xy if element not in '']
        x_coords = [int(element) for element in xy[::2] if element not in '']
        y_coords = [int(element) for element in xy[1::2] if element not in '']
        xcoords_jint = JArray(JInt)(x_coords)
        ycoords_jint = JArray(JInt)(y_coords)
        polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
        roi_instance = scyjava.jimport('ij.gui.Roi')
        imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
        imp.setRoi(imported_polygon)
        rm.addRoi(imported_polygon)
    ij.py.run_macro("roiManager('Select All');")
    rm.runCommand("Save", f"{voronoi_path}/" + f"{f_name}.zip")
    ij.py.run_macro("roiManager('Select All');")
    rm.runCommand("Delete")

In [ ]:
for file_path in file_list:
    image = ij.io().open(file_path)
    f_name = os.path.basename(file_path)
    f_name = os.path.splitext(f_name)[0]
    input_ROI = f"{voronoi_path}/{f_name}.zip"
    rm.open(input_ROI)
    ij.ui().show(image)
    #to better draw, apply and save ROIs, the image should be displayed:
    imp = ij.py.to_imageplus(image) 
    roi_count = rm.getCount()
    for i in range(roi_count):
        rm.select(i)
        shrinking = ij.IJ.run("Enlarge...", "enlarge=-1 pixel")
        rm.runCommand("update")
    rm.runCommand("Select All")
    rm.runCommand("XOR")
    rm.runCommand("Fill")
    rm.runCommand("Select All")
    rm.runCommand("XOR")
    ij.IJ.run("Clear Outside")
    rm.runCommand("Select All")
    rm.runCommand("Delete")
    ij.py.run_macro('setAutoThreshold("Default dark no-reset");')
    ij.IJ.run("Threshold...")
    ij.py.run_macro('setThreshold(5, 255);')
    ij.py.run_macro('setOption("BlackBackground", true);')
    ij.IJ.run("Convert to Mask", "black")
    
    voronoi = """
run("Set Measurements...", "center redirect=None decimal=1");
run("Analyze Particles...","size=3-Infinity display clear");
//Resolution de l'image pwidth et pheight
getPixelSize(unit, pw, ph, pd);
//Voronio
run("Voronoi");
setThreshold(0, 0,"black & white");

//Wand to ROI Manager
x=newArray(nResults);
y=newArray(nResults);
nbPoints=nResults;

for (i=0; i<nbPoints; i++) {
    x[i]=getResult("XM",i)/pw;
    y[i]=getResult("YM",i)/ph;
}
for (i=0; i<nbPoints; i++) {
    doWand(x[i], y[i], 156.0, "Legacy");
    roiManager("Add");
}

//Center of mass
/*
x=newArray(nResults);
y=newArray(nResults);
for (i=0; i<nResults; i++) {
    x[i]=getResult("XM",i)/pw;
    y[i]=getResult("YM",i)/ph;
}
*/
makeSelection("point", x, y);
selectWindow("Results");
run("Close");
close("*");
"""
    run_voronoi = ij.py.run_macro(voronoi)
    ij.py.run_macro("roiManager('Select All');")
    rm.runCommand("Save", f"{voronoi_path}/" + f"{f_name}_voronoi.zip")
    rm.runCommand("Delete")

In [ ]:
file_pattern = os.path.join(bact_path, "*.tif")
file_list = glob.glob(file_pattern)
for file_path in file_list:
    image = ij.io().open(file_path)
    f_name = os.path.basename(file_path)
    f_name = os.path.splitext(f_name)[0]
    ij.ui().show(image)
    ij.IJ.run("Smooth", "stack")
    ij.IJ.run("Smooth", "stack")
    roi_collection = """
    setAutoThreshold("Default dark no-reset");
    run("Threshold...");
    setThreshold(20, 255);
    setOption("BlackBackground", true);
    run("Convert to Mask", "black");  
    run("Analyze Particles...", "size=10-Infinity add stack");
    run("Set Measurements...", "area centroid stack redirect=None decimal=2");
    nbArea=roiManager("count")
    for (i=0; i<nbArea; i++) {
    	roiManager("Select", i);
    	run("Measure");	
    }
    close("*");
    //close();
    """
    rois = ij.py.run_macro(roi_collection)
    rm.runCommand("Select All")
    rm.runCommand("Save", f"{bact_path}/" + f"{f_name}_bactROI.zip") # this saves the ROIs as a zip file
    rm.runCommand("Delete")
    output_path = f"{bact_path}/{f_name}_bactROI.csv" # save giving a name matching the opened image
    saving = ij.IJ.saveAs("Results", output_path)
    ij.IJ.run("Clear Results")

In [ ]:
dsettings = { # specify parameters for the detection step in Trackmate (Threshold Detector)
    'TARGET_CHANNEL' : ij.py.to_java(1),
    'SIMPLIFY_CONTOURS' : False,
    'INTENSITY_THRESHOLD' : 20.0,
}
quality_filter = 50
tsettings = { # specify parameters for the tracking step in Trackmate (LAP Tracker)
    'LINKING_MAX_DISTANCE' : 100.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 45.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
}

In [ ]:
track_bacteria(bact_path, directory_path, dsettings, quality_filter, tsettings, False)

In [ ]:
grouped = merge_trackmate_rois(bact_path, directory_path)

In [ ]:
apply_bact_overlays(bact_path, grouped)

In [ ]:
print(file_list)

In [7]:
channels_image = directory_path+'channels.tif'
max_channels = directory_path+'max_channels.tif'
sum_channels = directory_path+'sum_channels.tif'
tracks_csv = directory_path+'da_Tracks.csv'
sorted_tm_tracks_csv = directory_path+'exportTracks_edited.csv'
tm_tracks_csv = directory_path+'exportTracks.csv'
roiset = directory_path+'rois/'
bact_path = directory_path+'bact/'
other = directory_path+'other/'
grouped = directory_path+'grouped/'
measurements = directory_path+'measurement/'

In [ ]:
run("Set Measurements...", "mean redirect=None decimal=4");
n=roiManager("count")
for (i=0; i<n; i++) {
		roiManager("Select", i);
		run("Enlarge...", "enlarge=5 pixel");
		roiManager("Add");
roiIndexes = newArray(i,n+i);
		roiManager("Select", roiIndexes);
		roiManager("XOR");
		run("Measure");	
	}

In [29]:
def circlesofhell(bact_path, bact_roi_path, file_path, output_path):
    roi_path = f"{bact_roi_path}/{file_path}"
    roi_pattern = os.path.join(bact_path, "*.roi")
    roi_list = glob.glob(roi_pattern)
    for roi in roi_list:
        open_roi = rm.open(roi)
        rm.select(0)
        enlarged = ij.IJ.run("Enlarge...", "enlarge=1 pixel")
        rm.addRoi(enlarged)
        for n in range(0,3):
            rm.select(1+n)
            enlarged = ij.IJ.run("Enlarge...", "enlarge=1 pixel")
            rm.addRoi(enlarged)
        rm.select(0)
        ij.IJ.run("Measure")
        ij.IJ.run("Make Band...", "band=1")
        ij.IJ.run("Measure")
        for n in range(0,4):
            rm.select(1+n)
            ij.IJ.run("Make Band...", "band=1")
            ij.IJ.run("Measure")
        rm.runCommand("Reset")
        saving = ij.IJ.saveAs("Results", output_path)
        ij.IJ.run("Clear Results")    

In [14]:
file_pattern = os.path.join(bact_path, "*channel.tif")
file_list = glob.glob(file_pattern)
for file_path in file_list: # cycle through images, open them and open the matching "grouped' csv file
    basename = os.path.basename(file_path)
    corename = os.path.splitext(basename)[0]
    bact_roi_path = bact_path + f"rois_bact/{corename}"
    if not os.path.exists(bact_roi_path):
        os.makedirs(bact_roi_path)
    imp = ij.IJ.openImage(file_path)
    ij.ui().show(imp)
    f_name = os.path.basename(file_path)
    f_name = os.path.splitext(f_name)[0]
    input_csv = Path(f"{grouped}/{f_name}_grouped.csv")
    df = pandas.read_csv(input_csv)
    pouet = df['object_ID_list']
    input_ROI = Path(f"{bact_path}/{f_name}_bactROI.zip")
    rm.open(f"{input_ROI}")
    for i in range(len(pouet)):
        single_row = df.iloc[i]
        row_cellids = single_row.object_ID_list
        row_cleaned = row_cellids.strip('[').strip(']')
        row_array = row_cleaned.split(', ')
        for cell in row_array:
            cell_index = int(cell)
            roi = rm.select(cell_index)
            output = f"{bact_roi_path}/{cell_index}.roi"
            saved = rm.runCommand("Save", output)
    rm.runCommand("Reset")
    ij.py.run_macro('close("*");')

In [ ]:
for file_path in file_list:
    basename = os.path.basename(file_path)
    corename = os.path.splitext(basename)[0]
    output_path = f"{bact_path}/{corename}_bactrings.csv"
    circlesofhell(bact_path, bact_roi_path, file_path, output_path)

In [28]:
roi_path = f"{bact_path}/rois_bact/Experiment-1546_bact_channel"
list = os.listdir(roi_path)
print(list)